#  --------------------------  FINAL MODEL  --------------------------

# Model Building Using Distilbert

###  Importing Libraries 

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline 

###   Loading the cleaned dataset 

In [2]:
email = pd.read_csv("new_email.csv")
email

,CONTENT,CLASS
0,eat shit john j lavorato john arnold football ...,Abusive
1,fuck,Abusive
2,gentleman following champagne available approx...,Abusive
3,sorry taken long trying fend chick life sooooo...,Abusive
4,asshole john j lavorato john arnold john cant ...,Abusive
...,...,...
24651,need new lead business need increased internet...,Non Abusive
24652,garth brook wrapped win ticket trip garth broo...,Non Abusive
24653,dear smartreminders member franchoice free one...,Non Abusive
24654,kid rock forever official kid rock site kidroc...,Non Abusive


###  labeling Abusive class as 0 and Non Abusive class as 1

In [3]:
email['LABEL'] = email['CLASS'].map({'Abusive': 0, 'Non Abusive': 1})
email

,CONTENT,CLASS,LABEL
0,eat shit john j lavorato john arnold football ...,Abusive,0
1,fuck,Abusive,0
2,gentleman following champagne available approx...,Abusive,0
3,sorry taken long trying fend chick life sooooo...,Abusive,0
4,asshole john j lavorato john arnold john cant ...,Abusive,0
...,...,...,...
24651,need new lead business need increased internet...,Non Abusive,1
24652,garth brook wrapped win ticket trip garth broo...,Non Abusive,1
24653,dear smartreminders member franchoice free one...,Non Abusive,1
24654,kid rock forever official kid rock site kidroc...,Non Abusive,1


###  Checking for null values 

In [4]:
email.isnull().sum()

CONTENT    124
CLASS        0
LABEL        0
dtype: int64

In [5]:
email.dropna(axis=0, inplace=True)

In [6]:
email.isnull().sum()

CONTENT    0
CLASS      0
LABEL      0
dtype: int64

In [7]:
x = list(email['CONTENT'])
y = list(email['LABEL'])

### Splitting the dataset as train and test

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x,y, test_size = 0.20, random_state = 0)

In [9]:
!pip install transformers 

     |████████████████████████████████| 2.6 MB 8.8 MB/s 
     |████████████████████████████████| 895 kB 73.4 MB/s 
     |████████████████████████████████| 3.3 MB 48.1 MB/s 
     |████████████████████████████████| 636 kB 59.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [10]:
import transformers 

In [11]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [12]:
train_encodings = tokenizer(X_train, truncation=True, padding=True)
test_encodings = tokenizer(X_test, truncation=True, padding=True)

In [13]:
import tensorflow as tf

train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    y_train
))

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    y_test
))

### Model Building 

In [14]:
from transformers import TFDistilBertForSequenceClassification

model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')

optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
model.compile(optimizer=optimizer, loss=model.compute_loss, metrics=['accuracy'])
model.fit(train_dataset.shuffle(len(X_train)).batch(16), epochs=3, batch_size=16)

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_projector', 'activation_13', 'vocab_transform', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier', 'dropout_19', 'pre_classifier']
You should probably TRAIN this model on a down-stream task to be able to use i

Epoch 1/3
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autogr

### Model Evaluation 

In [15]:
model.evaluate(test_dataset.shuffle(len(X_test)).batch(16), return_dict=True, batch_size=16)

307/307 [==============================] - 95s 305ms/step - loss: 0.1789 - accuracy: 0.9488


{'accuracy': 0.9488486051559448, 'loss': 0.17888770997524261}

### Saving the model

In [16]:
abusive_model = "/save_models" 

model.save_pretrained(abusive_model)
tokenizer.save_pretrained(abusive_model)

('/save_models/tokenizer_config.json',
 '/save_models/special_tokens_map.json',
 '/save_models/vocab.txt',
 '/save_models/added_tokens.json',
 '/save_models/tokenizer.json')